In [7]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeVectorStore
from pprint import pprint
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')
os.environ["LANGSMITH_API_KEY"] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [9]:
import sys
import importlib

sys.path.append('/Users/nishanshehadeh/GitHubProjects/policy-bot-overview/aviary/backend')
from myapp.rag_system import RAGSystem
importlib.reload(sys.modules['myapp.rag_system'])


<module 'myapp.rag_system' from '/Users/nishanshehadeh/GitHubProjects/policy-bot-overview/aviary/backend/myapp/rag_system.py'>

In [10]:
rag_system = RAGSystem()

In [11]:
res = []
res = rag_system.handle_search_query("War in Ukraine")

In [13]:
print(len(res))

6


In [14]:
res = rag_system.handle_chat_query("Give me some infomration on immigration")

In [15]:
print(len(res))

TypeError: object of type 'async_generator' has no len()

In [12]:
index_name = "langchain-index"

embeddings = OpenAIEmbeddings()
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)
llm = ChatOpenAI(model="gpt-4o")

In [40]:
old_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 6}
)

from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vector_store.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [48]:
result = retriever.invoke("Give me some info")
result2 = old_retriever.invoke("Give me some info")

In [49]:
print(result[0].metadata)
print(result2[0].metadata)

{'chunk_number': 16.0, 'document_id': '6758e2c33f07207cb38e7284', 'publication_date': '2024-12-05', 'source_url': 'https://www.govinfo.gov/content/pkg/FR-2024-12-05/pdf/2024-28437.pdf', 'title': 'Agency Information Collection Activities; Submission to the Office of Management and Budget; Federal Migratory Bird Hunting and Conservation Stamp (Duck Stamp) and Junior Duck Stamp Contests', 'score': 0.76227951}
{'chunk_number': 16.0, 'document_id': '6758e2c33f07207cb38e7284', 'publication_date': '2024-12-05', 'source_url': 'https://www.govinfo.gov/content/pkg/FR-2024-12-05/pdf/2024-28437.pdf', 'title': 'Agency Information Collection Activities; Submission to the Office of Management and Budget; Federal Migratory Bird Hunting and Conservation Stamp (Duck Stamp) and Junior Duck Stamp Contests'}


In [50]:
from langchain.retrievers.multi_query import MultiQueryRetriever

search_retriever = MultiQueryRetriever.from_llm(retriever, llm)


ValidationError: 1 validation error for MultiQueryRetriever
retriever
  value is not a valid dict (type=type_error.dict)

In [42]:
question = "What has happened with renwable energy or climate change?"
docs = search_retriever.invoke(question)


In [7]:
print(len(docs))

11


In [8]:
print(docs[0].metadata)


{'chunk_number': 1.0, 'document_id': '6758e2c63f07207cb38e72ad', 'publication_date': '2024-12-04', 'source_url': 'https://www.govinfo.gov/content/pkg/FR-2024-12-04/pdf/2024-28412.pdf', 'title': 'Combined Notice of Filings #1'}


In [3]:
import sys
import importlib

sys.path.append('/Users/nishanshehadeh/GitHubProjects/policy-bot-overview/policy-bot/backend')
from myapp.rag_system import RAGSystem
importlib.reload(sys.modules['myapp.rag_system'])

<module 'myapp.rag_system' from '/Users/nishanshehadeh/GitHubProjects/policy-bot-overview/policy-bot/backend/myapp/rag_system.py'>

In [11]:

rag_system = RAGSystem()
res = []
res = rag_system.handle_search_query("War in Ukraine")

In [9]:
print(len(res))

6


In [10]:
print(res)

{'6758e2ca3f07207cb38e72e3', '6758e2c83f07207cb38e72c8', '6758e2ca3f07207cb38e72e2', '6758e2c53f07207cb38e729d', '6758e2c53f07207cb38e72a1', '6758e2c53f07207cb38e729f'}
